In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import _stop_words
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [15]:
import string

In [5]:
try:
    from google.colab import files
    !wget -P DATAPATH https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Full-Economic-News-DFE-839861.csv
    !ls -lah DATAPATH
    our_data = pd.read_csv("DATAPATH/Full-Economic-News-DFE-839861.csv" , encoding = "ISO-8859-1" )

except ModuleNotFoundError:
    our_data = pd.read_csv("Data/Full-Economic-News-DFE-839861.csv" , encoding = "ISO-8859-1" )

--2023-12-05 23:51:55--  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch4/Data/Full-Economic-News-DFE-839861.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12383529 (12M) [text/plain]
Saving to: ‘DATAPATH/Full-Economic-News-DFE-839861.csv’

Full-Economic-News- 100%[===================>]  11.81M  --.-KB/s    in 0.1s    

2023-12-05 23:51:55 (84.7 MB/s) - ‘DATAPATH/Full-Economic-News-DFE-839861.csv’ saved [12383529/12383529]

total 12M
drwxr-xr-x 2 root root 4.0K Dec  5 23:51 .
drwxr-xr-x 1 root root 4.0K Dec  5 23:51 ..
-rw-r--r-- 1 root root  12M Dec  5 23:51 Full-Economic-News-DFE-839861.csv


In [6]:
our_data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...


In [7]:
our_data['relevance'].value_counts()

no          6571
yes         1420
not sure       9
Name: relevance, dtype: int64

In [8]:
our_data = our_data[our_data['relevance'] != 'not sure']

In [9]:
our_data['relevance'].value_counts()

no     6571
yes    1420
Name: relevance, dtype: int64

In [11]:
our_data['relevance'] = our_data['relevance'].map({'no':0,'yes':1})

In [12]:
our_data['relevance'].value_counts()

0    6571
1    1420
Name: relevance, dtype: int64

In [13]:
stopwords = _stop_words.ENGLISH_STOP_WORDS
print(stopwords)

frozenset({'myself', 'besides', 'three', 'seemed', 'herself', 'noone', 'same', 'could', 'down', 'of', 'somehow', 'show', 'sincere', 'all', 'anyway', 'thereafter', 'via', 'a', 'already', 'both', 'fifteen', 'had', 'empty', 'nevertheless', 'fire', 'meanwhile', 'where', 'by', 'whereby', 'these', 'latter', 'upon', 'cannot', 'something', 'move', 'detail', 'once', 'below', 'after', 'five', 'do', 'put', 'whereafter', 'the', 'last', 'under', 'as', 'enough', 'towards', 'done', 'itself', 'across', 'every', 'seeming', 'nine', 'amoungst', 'in', 'be', 'while', 'us', 'go', 'either', 'de', 'ltd', 'with', 'else', 'well', 'onto', 'during', 'sometimes', 'whom', 'how', 'beforehand', 'against', 'also', 'four', 'though', 'wherein', 'should', 'each', 'along', 'part', 'except', 'perhaps', 'twelve', 'this', 'throughout', 'around', 'have', 'yourself', 'not', 'that', 'often', 'first', 'or', 'because', 'within', 'himself', 'from', 'why', 'found', 'hundred', 'thick', 'here', 'is', 'ours', 'two', 're', 'etc', 'nobo

In [16]:
our_data['text'][0]

'NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.</br></br>On small-denomination, or "consumer," CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Banxquote Money Markets, a Wilmington, Del., information service.</br></br>On three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to Banxquote. Two banks in the Banxquote survey, Citibank in New York and CoreStates in Pennsylvania, are paying less than 5% on threemonth small-denomination CDs.</br></br>Declines were somewhat smaller on five-year consumer CDs, which eased to 7.37% from 7.45%, Banxquote said.</br></br>Yields on three-month and six-month Treasury bills sold at Monday\'s auction plummeted more than a fifth of a percentage po

In [18]:
def cleanup(doc):
  doc = doc.replace('</br>',' ')
  doc= "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
  doc = " ".join([token for token in doc.split() if token not in stopwords])
  return doc

In [19]:
print(cleanup("""
NEW YORK -- Yields on most certificates of deposit offered by major banks dropped more than a tenth of a percentage point in the latest week, reflecting the overall decline in short-term interest rates.</br></br>On small-denomination, or "consumer," CDs sold directly by banks, the average yield on six-month deposits fell to 5.49% from 5.62% in the week ended yesterday, according to an 18-bank survey by Banxquote Money Markets, a Wilmington, Del., information service.</br></br>On three-month "consumer" deposits, the average yield sank to 5.29% from 5.42% the week before, according to Banxquote. Two banks in the Banxquote survey, Citibank in New York and CoreStates in Pennsylvania, are paying less than 5% on threemonth small-denomination CDs.</br></br>Declines were somewhat smaller on five-year consumer CDs, which eased to 7.37% from 7.45%, Banxquote said.</br></br>Yields on three-month and six-month Treasury bills sold at Monday's auction plummeted more than a fifth of a percentage point from the previous week, to 5.46% and 5.63%, respectively.
"""))

NEW YORK Yields certificates deposit offered major banks dropped tenth percentage point latest week reflecting overall decline shortterm rates On smalldenomination consumer CDs sold directly banks average yield sixmonth deposits fell week ended yesterday according bank survey Banxquote Money Markets Wilmington Del information service On threemonth consumer deposits average yield sank week according Banxquote Two banks Banxquote survey Citibank New York CoreStates Pennsylvania paying threemonth smalldenomination CDs Declines somewhat smaller fiveyear consumer CDs eased Banxquote said Yields threemonth sixmonth Treasury bills sold Mondays auction plummeted fifth percentage point previous week respectively


In [20]:
from sklearn.model_selection import train_test_split
x = our_data.text
y = our_data.relevance


In [21]:
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [22]:
vect = CountVectorizer(preprocessor=cleanup)
x_train_dtm = vect.fit_transform(x_train)
x_test_dtm = vect.transform(x_test)

In [23]:
nb = MultinomialNB()
nb.fit(x_train_dtm,y_train)
y_pred = nb.predict(x_test_dtm)

In [25]:
print("Accuracy: ", accuracy_score(y_test, y_pred))

Accuracy:  0.7717717717717718


In [28]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(class_weight='balanced')
log_reg.fit(x_train_dtm,y_train)
y_pred = log_reg.predict(x_test_dtm)
print("Accuracy: ", accuracy_score(y_test, y_pred))

Accuracy:  0.7452452452452453


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
